# wokring with text data
[Token vizualize](https://tiktokenizer.vercel.app/)

In [2]:
with open("/content/the-verdict.txt",'r',encoding='utf-8') as f:
  data = f.read()

In [3]:
data[:100]

'I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no g'

In [4]:
len(data)

20479

In [5]:
import re

text = "I HAD always thought Jack Gisburn rather a cheap genius--though a"
result = re.split(r'([,.:;?_!"()\']|--|\s)', data)

In [6]:
result = [item for item in result if item.strip()]
preprocessed = result

In [7]:
len(preprocessed)

4690

# converting it the token to token id

In [8]:
all_words = list(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

1130


In [9]:
vocab = {vocab:interger for interger,vocab in enumerate(all_words)}

{'"lift': 0,
 'across': 1,
 'accuse': 2,
 'knew?': 3,
 'covered': 4,......}

In [10]:

class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)

        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [11]:
tokenizer = SimpleTokenizerV1(vocab)

In [12]:
text = "I HAD always thought Jack Gisburn rather a cheap genius--though a"
tokenizer.encode(text)

[920, 787, 1056, 449, 824, 159, 293, 573, 563, 540, 221, 1087, 573]

In [13]:
tokenizer.decode([675, 573, 805, 942, 433, 607, 759, 804, 939, 475, 210, 241, 804])

'among a somebody heard arms romantic embarrassed slowly stopped sex sugar mediocrity slowly'

In [14]:
preprocessed[:10]

['I',
 'HAD',
 'always',
 'thought',
 'Jack',
 'Gisburn',
 'rather',
 'a',
 'cheap',
 'genius']

In [15]:
all_tokens = sorted(list(set(preprocessed)))
all_words.extend(["<|endoftext|>","<junk|>"])

In [16]:
vocab = {token:interger for interger,token in enumerate(all_words)}

In [17]:
len(vocab.items())

1132

In [18]:
for i,item in enumerate(list(vocab.items())[-5:]):
  print(i,item)

0 ('glad', 1127)
1 ('where', 1128)
2 ('Rickham', 1129)
3 ('<|endoftext|>', 1130)
4 ('<junk|>', 1131)


In [19]:

class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)

        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        preprocessed = [
            item if item in self.str_to_int else "<junk|>" for item in preprocessed
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [20]:
tokenizer = SimpleTokenizerV2(vocab)

In [21]:
text1 = text + " Rohit"
tokenizer.encode(text1)

[920, 787, 1056, 449, 824, 159, 293, 573, 563, 540, 221, 1087, 573, 1131]

In [22]:
tokenizer.decode(tokenizer.encode(text1))

'I HAD always thought Jack Gisburn rather a cheap genius -- though a <junk|>'

# Byte Pair encoding

using tiktoken library to utilize the byte pair algo used in the gpt2

In [23]:
# !pip install tiktoken
import tiktoken

In [24]:
tokenizer = tiktoken.get_encoding("gpt2")

### here two word yet three encode (because it is using byte pair encoding)

In [25]:
tokenizer.encode("Hello rohit")

[15496, 686, 17945]

In [26]:
tokenizer.decode([15496, 686, 17945])

'Hello rohit'

In [27]:
tokenizer.encode("hello <|endoftext|>",allowed_special={'<|endoftext|>'})

[31373, 220, 50256]

# Data sampling with a sliding window

In [28]:
with open("/content/the-verdict.txt",'r',encoding='utf-8') as f:
  raw_text = f.read()

In [29]:
enc_text = tokenizer.encode(raw_text)
len(enc_text)

5145

In [30]:
gpt_tokenizer = tiktoken.get_encoding("gpt2")
context_size = 4
enc_sample = enc_text[:50]
for i in range(1,context_size+1):
  context = enc_sample[:i]
  desired = enc_sample[i]
  print(context," --> ",desired)
  print(gpt_tokenizer.decode(context)," --> ",gpt_tokenizer.decode([desired]))

[40]  -->  367
I  -->   H
[40, 367]  -->  2885
I H  -->  AD
[40, 367, 2885]  -->  1464
I HAD  -->   always
[40, 367, 2885, 1464]  -->  1807
I HAD always  -->   thought


In [31]:
import torch

In [32]:
 torch.__version__

'2.6.0+cu124'

In [33]:

from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [34]:

def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [35]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [36]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [37]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


# Token embedding

In [38]:
dir(tokenizer)[-5:]

['max_token_value',
 'n_vocab',
 'name',
 'special_tokens_set',
 'token_byte_values']

In [39]:
tokenizer.n_vocab

50257

In [40]:
input_ids = torch.tensor([1,2,3])
vocab_size = 6
output_size = 3

embedding_layer = torch.nn.Embedding(vocab_size, output_size)
embedding_layer.weight

Parameter containing:
tensor([[-0.8285,  0.9178, -2.1719],
        [ 0.2145, -0.1364,  0.7133],
        [-0.2867,  1.0596, -0.2108],
        [-2.0118, -2.0385, -0.4667],
        [-0.1621,  1.7422, -2.6219],
        [ 0.6709, -1.4519, -0.2321]], requires_grad=True)

In [41]:
output = embedding_layer(input_ids)
output

tensor([[ 0.2145, -0.1364,  0.7133],
        [-0.2867,  1.0596, -0.2108],
        [-2.0118, -2.0385, -0.4667]], grad_fn=<EmbeddingBackward0>)

# encoding word position

In [42]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [46]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)
max_length = 4
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


In [47]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [48]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [49]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [50]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
